In [1]:
import pandas as pd

In [2]:
tokens_df = pd.read_csv("results/tokens.tsv", names=["study", "notation", "slug", "token"], delimiter="\t")

In [71]:
study = "movies"
notation = "vega-lite"

filtered = tokens_df.query(f"study=='{study}' and notation=='{notation}'").groupby("token").nunique("slug")
max_count = filtered["slug"].max()
trie = dict()
for token, count in filtered["slug"].items():
    pointer = trie
    for c in token:
        if c not in pointer:
            pointer[c] = dict()
        pointer = pointer[c]
    pointer["count"] = count


In [72]:
from pathlib import Path

In [73]:
text = Path("studies/movies/vega-lite/bubble_agg.vl").read_text()

In [74]:
import spectra
import math
scale = spectra.scale([spectra.html("#F22"), spectra.html("#FFF")]).domain([math.log(1), math.log(max_count-1)])

out_text = ""

pointer = trie
buffer = ""
for c in text:
    if c not in pointer:
        out_text += "<span"
        if "count" in pointer and pointer["count"] < max_count:
            color = scale(math.log(pointer["count"])).hexcode
            out_text += f" style='background: {color}'"
            out_text += f" title='{pointer['count']}/{max_count}'"
        out_text += ">" + buffer + "</span>"
        
        buffer = ""
        pointer = trie
        out_text += c
    else:
        buffer += c
        pointer = pointer[c]

from IPython.core.display import HTML
HTML("<pre>" + out_text + "</pre>")
